In [31]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [33]:
df = pd.read_csv("../input/train-folds/train_folds.csv")
df_test = pd.read_csv("../input/tabular-playground-series-feb-2021/test.csv")
sample_sumbission = pd.read_csv("../input/tabular-playground-series-feb-2021/sample_submission.csv")

In [26]:
useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
df_test = df_test[useful_features]

In [32]:
final_prediction = []
for fold in range(5):
    xtrain = df[df.kfold!=fold].reset_index(drop=True)
    xvalid = df[df.kfold==fold].reset_index(drop=True)  
    xtest = df_test.copy()
    
    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]


    ordinal_encoder = OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    model = XGBRegressor(random_state=fold,n_jobs = 4)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    preds_test = model.predict(xtest)
    final_prediction.append(preds_test)
    print(fold, mean_squared_error(yvalid,preds_valid,squared= False))

In [39]:
preds = np.mean(np.column_stack(final_prediction), axis = 1)
preds

In [41]:
sample_sumbission.target = preds
sample_sumbission.to_csv("submission.csv", index = False)